### Github Config

In [52]:
!mkdir -p ~/.ssh
!ssh-keyscan github.com >> ~/.ssh/known_hosts
!chmod 600 ~/.ssh/known_hosts

# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16


In [53]:
!ls -al ~/.ssh

total 16
drwxr-xr-x 2 root root 4096 May 13 07:20 .
drwx------ 1 root root 4096 May 13 07:20 ..
-rw------- 1 root root  828 May 13 07:21 known_hosts


In [54]:
!ssh-keygen -t rsa -b 4096 -C "ivanjunqiwu99@gmail.com"

Generating public/private rsa key pair.
Enter file in which to save the key (/root/.ssh/id_rsa): 
Enter passphrase (empty for no passphrase): 
Enter same passphrase again: 
Your identification has been saved in /root/.ssh/id_rsa
Your public key has been saved in /root/.ssh/id_rsa.pub
The key fingerprint is:
SHA256:Ao8HEHaCeKsOodDmVas5UhYa9zoj1knFj3RxqW242OQ ivanjunqiwu99@gmail.com
The key's randomart image is:
+---[RSA 4096]----+
|..=...  ....     |
|o.o+o = ...      |
| o =o* = +       |
|o = ==+ = o      |
|o= *.=+=So       |
|+ = X...E        |
|o. o +           |
| .               |
|                 |
+----[SHA256]-----+


In [55]:
!cat ~/.ssh/id_rsa.pub

ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQCZlOxxY9Y536sPw6LdRYzOrA3ZEpX8Slc9WmtHQ83OHk/v04CFdGIscbI9l41F8OwsGM2Dlpl2e6XU1wmcY0Gvt6CQwB4e4f1yLR0MK9G5JD2UNoGeVtlb7fS47Ho6sYl9jp+MfSRj49IoXkngqS87IsnOZ0TfK9JQb5grsAtirYyNWxeYkKQrADzfaSCz/nLIsylHnNl+RqIU8wN4tinm3nm9taJbY4qr0qdolf7IrI93qb39gfOV863mz3KHMgpALdheejPIfJUVVZ/x7WAskyLiXDUfkRQAfSEoTDzo2LK88jqb4Uv+k2DTxD8U5UHMSsiYyYkYngZl1YjP0pgGH9rAdavV/Q5wAOJ3klUzRt6lg4CGuNEA5SZOBZTi3Kuyzct37oh4oXZ1493ZlHgATS9pNGsGHacx6IVRWwKFPiRi63u7A/Af7yQI20whfy1tcpEKNHoEwznQZEYp5dt/3VB3M+x/Rir/6+PG4m8bcd6UGpNwRiGoZ9fQgT9Lc1m3ddEukyraJ/84qxAsL/ELGrLQqYpRNbwaC92mNU2+N3JpEtiHBoeD+jcxlacms9D8M3MenFFhb1uY9Q9gdQPeSABynj8rXIrahxGrnBAJ/T3TmN/Tka4cKRs6fS3zG74Dq3FuW7cYD3HMWUMnPy2j3aCS7zCBkkLuRHkFJw7cTQ== ivanjunqiwu99@gmail.com


In [56]:
!git clone git@github.com:ivanckng/CS610_AML.git

Cloning into 'CS610_AML'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 85 (delta 43), reused 45 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (85/85), 5.60 MiB | 3.14 MiB/s, done.
Resolving deltas: 100% (43/43), done.


### Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install xgboost
!pip install lightgbm

In [11]:
import pandas as pd
from sklearn import linear_model, metrics, preprocessing, tree, model_selection, feature_selection, ensemble
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from collections import Counter
import lightgbm as lgb

import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [12]:
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
try:
    WordNetLemmatizer().lemmatize('cats')
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4')
try:
    SentimentIntensityAnalyzer().polarity_scores("hello")
except LookupError:
    nltk.download('vader_lexicon')

### Loading training and tesing data

In [17]:
train_set = pd.read_csv('/content/drive/MyDrive/10 Github/CS610_AML/01 Assignment 1/train.csv')
test_set = pd.read_csv('/content/drive/MyDrive/10 Github/CS610_AML/01 Assignment 1/test.csv')

### Data Processing & EDA

In [18]:
train_set.head()
orgin_instrances = train_set.shape[0]

In [19]:
train_set.describe()

,favourites_count,followers_count,friends_count,id,statuses_count,average_tweets_per_day,account_age_days,target
count,26206.000000,2.620600e+04,2.620600e+04,2.620600e+04,2.620600e+04,26206.000000,26206.000000,26206.000000
mean,12159.971342,3.732983e+05,4.427275e+03,1.253306e+17,2.072485e+04,6.754636,3017.870831,0.335152
std,33628.948536,2.515996e+06,4.602204e+04,3.035728e+17,6.345569e+04,22.782745,1018.958572,0.472053
min,0.000000,0.000000e+00,0.000000e+00,5.860000e+02,0.000000e+00,0.000000,483.000000,0.000000
25%,347.000000,3.400000e+01,3.400000e+01,9.799366e+07,1.331000e+03,0.486250,2302.000000,0.000000
50%,2037.000000,3.540000e+02,2.920000e+02,3.997210e+08,4.143500e+03,1.464000,3218.000000,0.000000
75%,8751.750000,8.241500e+03,8.780000e+02,2.480241e+09,1.679150e+04,5.460000,3895.000000,1.000000
max,839929.000000,1.216415e+08,1.641962e+06,1.120974e+18,2.771910e+06,1191.286000,5151.000000,1.000000


In [16]:
tweets_outliers = train_set[train_set['average_tweets_per_day']>= 100].shape[0]/train_set.shape[0]
print(f"outlier_percent:{tweets_outliers}")
friends_outliers = train_set[train_set['friends_count']  >= 10000].shape[0]/train_set.shape[0]
print(f"outlier_percent:{friends_outliers}")
followers_outliers = train_set[train_set['followers_count']  >= 3000000].shape[0]/train_set.shape[0]
print(f"outlier_percent:{followers_outliers}")
favorite_outliers = train_set[train_set['favourites_count']  >= 100000].shape[0]/train_set.shape[0]
print(f"outlier_percent:{favorite_outliers}")

outlier_percent:0.0066778600320537285
outlier_percent:0.02942074334121957
outlier_percent:0.025108753720522018
outlier_percent:0.023048156910631154


In [8]:
train_set = train_set[train_set['average_tweets_per_day']  <100]
train_set = train_set[train_set['friends_count']  < 10000]
train_set = train_set[train_set['followers_count']  < 3000000]
train_set = train_set[train_set['favourites_count']  < 100000]

In [9]:
train_set.describe()

,favourites_count,followers_count,friends_count,id,statuses_count,average_tweets_per_day,account_age_days,target
count,24194.000000,2.419400e+04,24194.000000,2.419400e+04,24194.000000,24194.000000,24194.000000,24194.000000
mean,7998.008184,1.352472e+05,670.164008,1.321844e+17,14558.937918,4.656627,2986.549764,0.346408
std,15300.520927,4.287107e+05,1166.772529,3.105272e+17,30939.644511,9.189648,1023.881887,0.475835
min,0.000000,0.000000e+00,0.000000,1.020200e+04,0.000000,0.000000,483.000000,0.000000
25%,330.000000,2.800000e+01,24.000000,1.069260e+08,1185.000000,0.436000,2252.000000,0.000000
50%,1925.000000,2.630000e+02,258.000000,4.247580e+08,3606.000000,1.275000,3185.000000,0.000000
75%,7303.500000,3.098250e+03,764.000000,2.592261e+09,13808.000000,4.487500,3863.000000,1.000000
max,99732.000000,2.996510e+06,9997.000000,1.120974e+18,445331.000000,99.738000,5048.000000,1.000000


In [10]:
new_instances = train_set.shape[0]
print(f"instances_removed:{new_instances/orgin_instrances}")

instances_removed:0.9232236892314737


In [11]:
test_set.describe()

,index,favourites_count,followers_count,friends_count,id,statuses_count,average_tweets_per_day,account_age_days
count,11232.000000,11232.000000,1.123200e+04,1.123200e+04,1.123200e+04,1.123200e+04,11232.000000,11232.000000
mean,5615.500000,12633.582176,3.633372e+05,4.489438e+03,1.147413e+17,2.188658e+04,7.211383,3032.060808
std,3242.543446,34600.722428,2.362189e+06,5.692586e+04,2.928501e+17,7.559739e+04,28.332281,1001.284291
min,0.000000,0.000000,0.000000e+00,0.000000e+00,4.180000e+02,0.000000e+00,0.000000,500.000000
25%,2807.750000,397.000000,3.700000e+01,4.375000e+01,1.019660e+08,1.422750e+03,0.511750,2345.000000
50%,5615.500000,2132.000000,3.900000e+02,3.060000e+02,3.969268e+08,4.392500e+03,1.553000,3222.000000
75%,8423.250000,9221.250000,8.979500e+03,9.232500e+02,2.403527e+09,1.800525e+04,5.676750,3879.000000
max,11231.000000,885123.000000,1.084423e+08,4.343060e+06,1.114681e+18,2.497571e+06,1269.055000,5158.000000


In [20]:
print("Size of Training Instances:", train_set.shape[0])
print("Sice of Features:", train_set.shape[1])
print(train_set.info())

print("Size of Testing Instances:", test_set.shape[0])
print("Sice of Features:", test_set.shape[1])
print(test_set.info())

Size of Training Instances: 26206
Sice of Features: 19
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26206 entries, 0 to 26205
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   created_at                    26206 non-null  object 
 1   default_profile               26206 non-null  bool   
 2   default_profile_image         26206 non-null  bool   
 3   description                   21115 non-null  object 
 4   favourites_count              26206 non-null  int64  
 5   followers_count               26206 non-null  int64  
 6   friends_count                 26206 non-null  int64  
 7   geo_enabled                   26206 non-null  bool   
 8   id                            26206 non-null  int64  
 9   lang                          20618 non-null  object 
 10  location                      26204 non-null  object 
 11  profile_background_image_url  22971 non-null  object 
 12  profi

In [21]:
train_set.drop(columns=['created_at','id', 'profile_background_image_url', 'profile_image_url', 'screen_name', 'location'], inplace=True)
test_set.drop(columns=['created_at','id', 'profile_background_image_url', 'profile_image_url', 'screen_name', 'location'], inplace=True)

In [22]:
# train_set['lang'] = train_set['lang'].isna().astype(int)
# train_set['lang'] = np.where(train_set['lang'] == 1, 0, 1)
train_set['lang'] = train_set['lang'].fillna('unknown')
# test_set['lang'] = test_set['lang'].isna().astype(int)
# test_set['lang'] = np.where(test_set['lang'] == 1, 0, 1)
test_set['lang'] = test_set['lang'].fillna('unknown')

In [23]:
train_set['default_profile'] = train_set['default_profile'].astype(int)
train_set['default_profile_image'] = train_set['default_profile_image'].astype(int)
train_set['geo_enabled'] = train_set['geo_enabled'].astype(int)
train_set['verified'] = train_set['verified'].astype(int)

test_set['default_profile'] = test_set['default_profile'].astype(int)
test_set['default_profile_image'] = test_set['default_profile_image'].astype(int)
test_set['geo_enabled'] = test_set['geo_enabled'].astype(int)
test_set['verified'] = test_set['verified'].astype(int)

In [24]:
train_set['follower_ratio'] = train_set['followers_count'] / (train_set['friends_count']+1)
test_set['follower_ratio'] = test_set['followers_count'] / (test_set['friends_count']+1)
train_set['favourite_per_tweet'] = train_set['favourites_count'] / (train_set['statuses_count']+1)
test_set['favourite_per_tweet'] = test_set['favourites_count'] / (test_set['statuses_count']+1)
train_set['favourite_per_day'] = train_set['favourites_count'] / (train_set['average_tweets_per_day']+1)
test_set['favourite_per_day'] = test_set['favourites_count'] / (test_set['average_tweets_per_day']+1)

In [25]:
train_set['lang'] = train_set['lang'].astype('category')
test_set['lang'] = test_set['lang'].astype('category')

In [29]:
train_set.drop(columns=['statuses_count'], inplace=True)
test_set.drop(columns=['statuses_count'], inplace=True)

### NLP

In [30]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if pd.isna(text) or text == "":
        return ""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

train_set['description'] = train_set['description'].fillna('')
train_set['description'] = train_set['description'].apply(preprocess_text)
test_set['description'] = test_set['description'].fillna('')
test_set['description'] = test_set['description'].apply(preprocess_text)

In [31]:
train_set['desc_len'] = train_set['description'].apply(len)
train_set['desc_wordcount'] = train_set['description'].apply(lambda x: len(x.split()))


test_set['desc_len'] = test_set['description'].apply(len)
test_set['desc_wordcount'] = test_set['description'].apply(lambda x: len(x.split()))

In [32]:
sid = SentimentIntensityAnalyzer()
def get_sentiment_scores(text):
    if pd.isna(text) or text == "":
        return 0.0
    return sid.polarity_scores(text)['compound']

train_set['desc_sentiment'] = train_set['description'].apply(get_sentiment_scores)
test_set['desc_sentiment'] = test_set['description'].apply(get_sentiment_scores)

#### Data Split

In [33]:
train_set = train_set.drop(columns=['description'])
test_set = test_set.drop(columns=['description'])

In [34]:
num_majority = train_set[train_set['target'] == 0].shape[0]
num_minority = train_set[train_set['target'] == 1].shape[0]

In [35]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(train_set.drop(columns=['target']), train_set['target'], test_size=0.2, shuffle = True, random_state=42, stratify=train_set['target'])

In [36]:
counter = Counter(y_train)
estimate_scale_pos_weight = counter[0] / counter[1]

In [37]:
train_predictors = train_set.drop(columns=['target'])
train_target = train_set['target']

In [38]:
counter_train = Counter(train_target)
estimate_scale_pos_weight_train = counter_train[0] / counter_train[1]

In [39]:
x_train

,default_profile,default_profile_image,favourites_count,followers_count,friends_count,geo_enabled,lang,verified,average_tweets_per_day,account_age_days,follower_ratio,favourite_per_tweet,favourite_per_day,desc_len,desc_wordcount,desc_sentiment
16919,0,0,1534,45,222,0,pt,0,0.705,3915,0.201794,0.555797,899.706745,77,17,0.0000
3662,0,0,2207,11557,820,1,en,0,1.984,3820,14.076736,0.291238,739.611260,33,6,0.0000
6669,0,0,738,36684,185,1,ar,0,42.414,3328,197.225806,0.005228,16.999125,46,6,0.0000
7412,0,0,24553,970,968,1,en,0,2.783,3703,1.001032,2.382399,6490.351573,27,3,0.0000
8506,0,0,8942,1898,807,1,en,1,2.601,4170,2.349010,0.824223,2483.199111,64,9,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,0,0,2768,295,295,1,en,0,4.086,3394,0.996622,0.199567,544.239088,13,3,0.0762
92,0,0,63616,883,742,1,it,0,25.954,2013,1.188425,1.217601,2360.169177,20,4,0.0000
8298,0,0,3256,341,186,1,en,0,0.386,4092,1.823529,2.059456,2349.206349,63,11,0.4215
14834,1,0,1332,8,31,0,en,0,0.912,1462,0.250000,0.998501,696.652720,21,2,0.0000


### Random Forest

In [ ]:
rf_model = ensemble.RandomForestClassifier(
    random_state=42,
    class_weight='balanced_subsample',
)

In [ ]:
param_grid_rf = {
    'n_estimators': [100, 200, 300, 400, 500],
    'criterion': ['gini', 'entropy'],
    'max_features': ['sqrt', 'log2']
}

In [ ]:
kfold = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
grid_search_rf = model_selection.GridSearchCV(estimator=rf_model,
                              param_grid=param_grid_rf,
                              cv=kfold,
                              scoring='roc_auc',
                              n_jobs=-1,
                              verbose=2
                             )

In [ ]:
grid_search_rf.fit(x_train, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc', verbose=2)

In [ ]:
best_rf_model = grid_search_rf.best_estimator_

In [ ]:
y_train_pred_proba_rf = best_rf_model.predict_proba(x_train)[:, 1]
train_auc_rf = metrics.roc_auc_score(y_train, y_train_pred_proba_rf)
print(f"Train AUC for tuned Random Forest: {train_auc_rf:.4f}")
y_test_pred_proba_rf = best_rf_model.predict_proba(x_test)[:, 1]
test_auc_rf = metrics.roc_auc_score(y_test, y_test_pred_proba_rf)
print(f"Test AUC for tuned Random Forest: {test_auc_rf:.4f}")

Train AUC for tuned Random Forest: 1.0000
Test AUC for tuned Random Forest: 0.9413


In [ ]:
train_predictors = train_set.drop(columns=['target'])
train_target = train_set['target']

In [ ]:
grid_search_rf.fit(train_predictors, train_target)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='roc_auc', verbose=2)

In [ ]:
best_rf_model = grid_search_rf.best_estimator_

In [ ]:
train_target_pred_proba_rf = best_rf_model.predict_proba(train_predictors)[:, 1]
train_auc_rf = metrics.roc_auc_score(train_target, train_target_pred_proba_rf)
print(f"Train AUC for tuned Random Forest: {train_auc_rf:.4f}")

Train AUC for tuned Random Forest: 1.0000


In [ ]:
test_set_predictors = test_set.drop(columns=['index'])
test_set_target_proba = best_rf_model.predict_proba(test_set_predictors)[:, 1]

test_set_prob = pd.DataFrame({'index': test_set['index'], 'target': test_set_target_proba})
test_set_prob.to_csv('rf_test_set_prob.csv', index=False)

### XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(use_label_encoder=False,
eval_metric='logloss',
random_state=42,
tree_method='hist',
device='cuda',
scale_pos_weight=estimate_scale_pos_weight,
enable_categorical = True
)  # Important:  Set eval_metric

In [ ]:
param_grid_xgb = {
    'n_estimators': [500,600,700,800,900,1000],
    'learning_rate': [0.001, 0.01, 0.02]
}

In [ ]:
grid_search_xgb = model_selection.GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgb,
    scoring='roc_auc',
    cv=10,
    n_jobs=-1,
    verbose=1
)

In [ ]:
grid_search_xgb.fit(x_train, y_train)

Fitting 10 folds for each of 18 candidates, totalling 180 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:29:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=True,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rat...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02],
                         'n_estimators': [500, 600, 700, 800, 900, 1000]},
             scoring='roc_auc', verbose=1)

In [ ]:
best_xgb_model = grid_search_xgb.best_estimator_
y_pred_train_proba_xgb = best_xgb_model.predict_proba(x_train)[:, 1]
train_auc_xgb = metrics.roc_auc_score(y_train, y_pred_train_proba_xgb)
print(f"Train AUC for tuned XGBoost: {train_auc_xgb:.4f}")
y_pred_test_proba_xgb = best_xgb_model.predict_proba(x_test)[:, 1]
test_auc_xgb = metrics.roc_auc_score(y_test, y_pred_test_proba_xgb)
print(f"Test AUC for tuned XGBoost: {test_auc_xgb:.4f}")

Train AUC for tuned XGBoost: 0.9811
Test AUC for tuned XGBoost: 0.9406


In [ ]:
# Original XGBoost model
grid_search_xgb.fit(train_predictors, train_target)

Fitting 10 folds for each of 18 candidates, totalling 180 fits


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [12:37:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device='cuda',
                                     early_stopping_rounds=None,
                                     enable_categorical=True,
                                     eval_metric='logloss', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rat...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02],
                         'n_estimators': [500, 600, 700, 800, 900, 1000]},
             scoring='roc_auc', verbose=1)

In [ ]:
best_xgb_model = grid_search_xgb.best_estimator_
train_target_proba = best_xgb_model.predict_proba(train_predictors)[:, 1]
train_auc_xgb = metrics.roc_auc_score(train_target, train_target_proba)
print(f"Train AUC for tuned XGBoost: {train_auc_xgb:.4f}")

Train AUC for tuned XGBoost: 0.9764


In [ ]:
test_set_predictors = test_set.drop(columns=['index'])
test_set_target_proba = best_xgb_model.predict_proba(test_set_predictors)[:, 1]

test_set_prob = pd.DataFrame({'index': test_set['index'], 'target': test_set_target_proba})
test_set_prob.to_csv('/content/drive/MyDrive/10 Github/CS610_AML/xgb_test_set_prob.csv', index=False)

In [ ]:
best_xgb_model.feature_importances_

array([0.03113176, 0.0092409 , 0.06154347, 0.17197716, 0.04745121,
       0.07212621, 0.00968822, 0.03960389, 0.40006047, 0.05895726,
       0.0436013 , 0.01872987, 0.02009667, 0.01579155], dtype=float32)

### LGB

In [40]:
lgbm_classifier = lgb.LGBMClassifier(
    objective='binary',
    metric='auc',
    n_estimators=500,
    learning_rate=0.1,
    num_leaves=31,
    max_depth=-1,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    device='gpu',
    categorical_feature=['lang']
)

In [41]:
cv_strategy = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [42]:
param_grid_lgbm = {
    'n_estimators': [400, 500, 600],
    'learning_rate': [0.001, 0.01, 0.02],
    'num_leaves': [15, 30, 45]

}

In [43]:
cv_strategy = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42) # CV折数可以调整，3折会快一些

In [44]:
grid_search_lgb = model_selection.GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid_lgbm,
    scoring='roc_auc',
    cv=cv_strategy,
    n_jobs=1,
    verbose=2
)

In [46]:
grid_search_lgb.fit(x_train,y_train)

Fitting 10 folds for each of 27 candidates, totalling 270 fits
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2508
[LightGBM] [Info] Number of data points in the train set: 18867, number of used features: 16
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.29 MB) transferred to GPU in 0.001135 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335135 -> initscore=-0.685049
[LightGBM] [Info] Start training from score -0.685049


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   1.0s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2508
[LightGBM] [Info] Number of data points in the train set: 18867, number of used features: 16
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.29 MB) transferred to GPU in 0.001039 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335135 -> initscore=-0.685049
[LightGBM] [Info] Start training from score -0.685049


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   1.1s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2508
[LightGBM] [Info] Number of data points in the train set: 18867, number of used features: 16
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.29 MB) transferred to GPU in 0.056721 secs. 1 sparse feature groups


...

[CV] END learning_rate=0.02, n_estimators=600, num_leaves=45; total time=   3.0s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6324, number of negative: 12544
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2514
[LightGBM] [Info] Number of data points in the train set: 18868, number of used features: 16
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.29 MB) transferred to GPU in 0.001153 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335171 -> initscore=-0.684891
[LightGBM] [Info] Start training from score -0.684891


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.02, n_estimators=600, num_leaves=45; total time=   3.0s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7026, number of negative: 13938
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2514
[LightGBM] [Info] Number of data points in the train set: 20964, number of used features: 16
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.32 MB) transferred to GPU in 0.001180 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335146 -> initscore=-0.685001
[LightGBM] [Info] Start training from score -0.685001


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=LGBMClassifier(categorical_feature=['lang'],
                                      colsample_bytree=0.8, device='gpu',
                                      metric='auc', n_estimators=500, n_jobs=-1,
                                      objective='binary', random_state=42,
                                      subsample=0.8),
             n_jobs=1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02],
                         'n_estimators': [400, 500, 600],
                         'num_leaves': [15, 30, 45]},
             scoring='roc_auc', verbose=2)

In [48]:
best_lgb_model = grid_search_lgb.best_estimator_
y_pred_train_proba_lgb= best_lgb_model.predict_proba(x_train)[:, 1]
train_auc_lgb = metrics.roc_auc_score(y_train, y_pred_train_proba_lgb)
print(f"Train AUC for tuned XGBoost: {train_auc_lgb:.4f}")
y_pred_test_proba_lgb = best_lgb_model.predict_proba(x_test)[:, 1]
test_auc_lgb = metrics.roc_auc_score(y_test, y_pred_test_proba_lgb)
print(f"Test AUC for tuned XGBoost: {test_auc_lgb:.4f}")

Train AUC for tuned XGBoost: 0.9799
Test AUC for tuned XGBoost: 0.9480


In [49]:
grid_search_lgb.fit(train_predictors, train_target)

Fitting 10 folds for each of 27 candidates, totalling 270 fits
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7904, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2511
[LightGBM] [Info] Number of data points in the train set: 23585, number of used features: 16
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.36 MB) transferred to GPU in 0.001261 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335128 -> initscore=-0.685081
[LightGBM] [Info] Start training from score -0.685081


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   1.1s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7904, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2520
[LightGBM] [Info] Number of data points in the train set: 23585, number of used features: 16
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.36 MB) transferred to GPU in 0.001361 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335128 -> initscore=-0.685081
[LightGBM] [Info] Start training from score -0.685081


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   1.1s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7904, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2518
[LightGBM] [Info] Number of data points in the train set: 23585, number of used features: 16
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.36 MB) transferred to GPU in 0.001252 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335128 -> initscore=-0.685081
[LightGBM] [Info] Start training from score -0.685081


...

[CV] END learning_rate=0.02, n_estimators=600, num_leaves=45; total time=   5.2s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7905, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2516
[LightGBM] [Info] Number of data points in the train set: 23586, number of used features: 16
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.36 MB) transferred to GPU in 0.014561 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335156 -> initscore=-0.684954
[LightGBM] [Info] Start training from score -0.684954


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.02, n_estimators=600, num_leaves=45; total time=   3.2s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 8783, number of negative: 17423
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2522
[LightGBM] [Info] Number of data points in the train set: 26206, number of used features: 16
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.40 MB) transferred to GPU in 0.001269 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335152 -> initscore=-0.684973
[LightGBM] [Info] Start training from score -0.684973


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=LGBMClassifier(categorical_feature=['lang'],
                                      colsample_bytree=0.8, device='gpu',
                                      metric='auc', n_estimators=500, n_jobs=-1,
                                      objective='binary', random_state=42,
                                      subsample=0.8),
             n_jobs=1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02],
                         'n_estimators': [400, 500, 600],
                         'num_leaves': [15, 30, 45]},
             scoring='roc_auc', verbose=2)

In [50]:
best_lgb_model = grid_search_lgb.best_estimator_
train_target_proba = best_lgb_model.predict_proba(train_predictors)[:, 1]
train_auc_lgb = metrics.roc_auc_score(train_target, train_target_proba)
print(f"Train AUC for tuned XGBoost: {train_auc_lgb:.4f}")

Train AUC for tuned XGBoost: 0.9802


In [51]:
test_set_predictors = test_set.drop(columns=['index'])
test_set_target_proba = best_lgb_model.predict_proba(test_set_predictors)[:, 1]

test_set_prob = pd.DataFrame({'index': test_set['index'], 'target': test_set_target_proba})
test_set_prob.to_csv('/content/drive/MyDrive/10 Github/CS610_AML/01 Assignment 1/lgb_test_set_prob_6.csv', index=False)